<a href="https://colab.research.google.com/github/gtgindia/-Segmentation-based-industrial-defect-detection/blob/main/1Wobot1_Train_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


In [ ]:
color_mode = "grayscale"
number_colour_layers = 1
image_size = (512, 512)
image_shape = image_size + (number_colour_layers,)

In [ ]:
image_shape

(512, 512, 1)

In [ ]:
training_data_path = "/content/drive/MyDrive/Wobot/WithDef/"
#test_data_path = "./casting_data/casting_data/test"
SEED = 42

In [ ]:
def get_image_data(data_path, color_mode, image_size, seed = None):

    return tf.keras.preprocessing.image_dataset_from_directory(
        data_path,
        color_mode=color_mode,
        image_size=image_size,
        seed=seed,
        batch_size=6
    )

In [ ]:
training_ds = get_image_data(
    training_data_path,
    color_mode,
    image_size,
    SEED
)

Found 780 files belonging to 6 classes.


In [ ]:
preprocessing_layers = [
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(512,512,1))
]

In [ ]:
def conv_2d_pooling_layers(filters, number_colour_layers):
    return [
        tf.keras.layers.Conv2D(
            filters,
            number_colour_layers,
            padding='same',
            activation='relu'
        ),
        tf.keras.layers.MaxPooling2D()
    ]
core_layers = \
    conv_2d_pooling_layers(16, number_colour_layers) + \
    conv_2d_pooling_layers(32, number_colour_layers) + \
    conv_2d_pooling_layers(64, number_colour_layers)

In [ ]:
class_names = training_ds.class_names
number_classes = len(class_names)
dense_layers = [
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(number_classes)
]

In [ ]:
model = tf.keras.Sequential(
    preprocessing_layers +
    core_layers +
    dense_layers
)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer='adam',
    loss=loss,
    metrics=['accuracy']
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='acc', patience=3, mode='auto'
)

In [ ]:

history = model.fit(
    training_ds,epochs = 4,callbacks = [callback]
)

Epoch 1/8
130/130 [==============================] - 677s 5s/step - loss: 3.0254 - accuracy: 0.4714
Epoch 2/8
130/130 [==============================] - 7s 50ms/step - loss: 0.0462 - accuracy: 0.9842
Epoch 3/8
130/130 [==============================] - 7s 50ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 4/8
130/130 [==============================] - 7s 50ms/step - loss: 7.1531e-04 - accuracy: 1.0000
Epoch 5/8
130/130 [==============================] - 7s 51ms/step - loss: 3.1633e-04 - accuracy: 1.0000
Epoch 6/8
130/130 [==============================] - 7s 50ms/step - loss: 1.8330e-04 - accuracy: 1.0000
Epoch 7/8
130/130 [==============================] - 7s 50ms/step - loss: 9.6761e-05 - accuracy: 1.0000
Epoch 8/8
130/130 [==============================] - 7s 50ms/step - loss: 7.0771e-05 - accuracy: 1.0000


In [ ]:
model.save("/content/drive/MyDrive/Wobot/Model1/2Wobot1_train_model.h5")